# Setup 

In [1]:
%%bash
# Install deps from 
# https://github.com/mwydmuch/ViZDoom/blob/master/doc/Building.md#-linux

apt-get update


apt-get install build-essential zlib1g-dev libsdl2-dev libjpeg-dev \
nasm tar libbz2-dev libgtk2.0-dev cmake git libfluidsynth-dev libgme-dev \
libopenal-dev timidity libwildmidi-dev unzip

# Boost libraries
apt-get install libboost-all-dev

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [92.1 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,540 kB]
Get:11 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
G

In [2]:
!pip install vizdoom
!pip install ray 
!pip install ray['rllib']
!pip install Ipython --upgrade


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.7 MB 8.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for vizdoom: filename=vizdoom-1.1.13-cp37-cp37m-linux_x86_64.whl size=14101146 sha256=21eb6666b66e7998afe58d52601225c6085298a35a98e6febb97a3dd5769ee6f
  Stored in directory: /root/.cache/pip/wheels/ac/37/ae/8e648023f66bb4c473701f94ce126032ff39ad9759ca0645a7
Successfully built vizdoom
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 59.4 MB 1.2 MB/s 
     |████████████████████████████████| 8.8 MB 53.7 MB/s 
     |████████████████████████████████| 4.1 MB 57.0 MB/s 
     |████████████████████████████████| 468 kB 56.6 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.48.1
    Uni

In [3]:
system_path = '/content/drive/MyDrive/GitHub/INM363-Project'
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append(system_path)

Mounted at /content/drive


In [4]:
from src.vizdoom_gym.envs.VizDoomEnv import VizdoomEnv
from src.vizdoom_gym.envs.VizDoomEnv_def import VizDoomVeryDenseReward

In [5]:
from ray.tune.registry import register_env
import gym
import os
import ray
import ray.rllib.agents.ppo as ppo
import shutil
import torch

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device: ", device, "\n")

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

device:  cuda:0 

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


# Initialize Ray

In [7]:
#need this to load vizdoom module 
system_path = '/content/drive/MyDrive/GitHub/INM363-Project/src' 
sys.path.append(system_path)

#need this to use gpu on ray 
os.environ['PYTHONPATH'] = '/content/drive/MyDrive/GitHub/INM363-Project' 
os.environ['PYTHONPATH']

'/content/drive/MyDrive/GitHub/INM363-Project'

In [8]:

chkpt_root = "/content/drive/MyDrive/GitHub/INM363-Project/model_checkpoints/ppo/easy_no_reward"
shutil.rmtree(chkpt_root, ignore_errors=True, onerror=None)


ray.shutdown()
print("Shutdown ray")

# start Ray -- add `local_mode=True` here for debugging
ray.init(ignore_reinit_error=True,  num_cpus =2, num_gpus = 1) #local_mode=True,

#ray.init(num_cpus= 2, num_gpus=1)

print("Initialized ray")

# register the custom environment
select_env = "VizDoomVeryDenseReward-v0"

register_env(select_env, lambda config: VizDoomVeryDenseReward())
#register_env(select_env, lambda config: VizdoomEnv())

print("registered environment")


Shutdown ray


2022-09-19 20:36:41,356	INFO worker.py:1518 -- Started a local Ray instance.


Initialized ray
registered environment


Training Config

In [9]:
# configure the environment and create agent
config = ppo.DEFAULT_CONFIG.copy()
config["log_level"] = "WARN"
config["num_workers"] = 1
config["framework"] = "torch"
config["model"] = {"dim": 42, 
                   "grayscale": True,
                   }
config["num_gpus"] = 1
config["preprocessor_pref"] = "rllib"
config['explore'] = True 
config['batch_mode'] = 'complete_episodes'


#config["horizon"] = 50
agent = ppo.PPOTrainer(config, env=select_env)

print("created agent")

2022-09-19 20:36:42,044	INFO ppo.py:379 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2022-09-19 20:36:42,048	INFO algorithm.py:354 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
/usr/local/lib/python3.7/dist-packages/ray/_private/ray_option_utils.py:273: DeprecationWarning: Setting 'object_store_memory' for actors is deprecated since it doesn't actually reserve the required object store memory. Use object spilling that's enabled by default (https://docs.ray.io/en/releases-2.0.0/ray-core/objects/object-spilling.html) instead to bypass the object store memory size limitation.
  stacklevel=1,


(RolloutWorker pid=8649) config file: /content/drive/MyDrive/GitHub/INM363-Project/scenarios/custom/very_dense_reward.cfg
(RolloutWorker pid=8649) scenario file: /content/drive/MyDrive/GitHub/INM363-Project/scenarios/custom/train/easy_no_reward_rs.wad
(RolloutWorker pid=8649) episode timeout: 400
(RolloutWorker pid=8649) screen resolution: 320X240


(RolloutWorker pid=8649) 2022-09-19 20:36:48,867	WARNING env.py:143 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.
2022-09-19 20:37:03,043	INFO trainable.py:164 -- Trainable.setup took 21.000 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2022-09-19 20:37:03,048	WARNING util.py:65 -- Install gputil for GPU system monitoring.


created agent


Training loop

In [10]:
import pandas as pd
import time 

cols = ["checkpoint", "eps_reward_min", "eps_reward_mean", "eps_reward_max", "eps_len_mean", "episodes_this_iter"]
results_df = pd.DataFrame(columns = cols) 

In [11]:
#chkpt_root = "/content/drive/MyDrive/GitHub/INM363-Project/model_checkpoints/ppo/dense"
#chkpt_file  = "/content/drive/MyDrive/GitHub/INM363-Project/model_checkpoints/ppo/dense/checkpoint_000190"
#agent.restore(chkpt_file)

In [ ]:
status = "{:2d} reward {:6.2f}/{:6.2f}/{:6.2f} len {:4.2f}"
start_n = 0
n_iter = 100

print("started training loop")
time_start = time.time() 

# train a policy with RLlib using PPO
for n in range(start_n, n_iter):
  
    result = agent.train()

    #print(i)
    #print(result)
    if (n+1) % 10 == 0 or n == 0: 
      chkpt_file = agent.save(chkpt_root)
      print(f"Saved checkpoint {n+1} at {chkpt_file}")
    #chkpt_file = "not saving checkpoints"

    print(status.format(
        n + 1,
        result["episode_reward_min"],
        result["episode_reward_mean"],
        result["episode_reward_max"],
        result["episode_len_mean"]
    ))

    #save metrics
    row = {'checkpoint': n+1,
       "eps_reward_min": result["episode_reward_min"],
       "eps_reward_mean": result["episode_reward_mean"],
       "eps_reward_max": result["episode_reward_max"],
       "eps_len_mean": result["episode_len_mean"],
       "episodes_this_iter": result["episodes_this_iter"]
       }
    results_df = results_df.append(row, ignore_index = True)


print(f"Total time elapsed: {(time.time()-time_start)/60}")

print("ending training loop")

ray.shutdown()
print("shutdown ray")

started training loop


(RolloutWorker pid=8649) 2022-09-19 20:37:05,339	WARNING deprecation.py:48 -- DeprecationWarning: `concat_samples` has been deprecated. Use `concat_samples() from rllib.policy.sample_batch` instead. This will raise an error in the future!


Saved checkpoint 1 at /content/drive/MyDrive/GitHub/INM363-Project/model_checkpoints/ppo/easy_no_reward/checkpoint_000001
 1 reward  -0.40/ -0.40/ -0.40 len 100.00
 2 reward  -0.40/ -0.40/ -0.40 len 100.00
 3 reward  -0.40/ -0.40/ -0.40 len 100.00
 4 reward  -0.40/ -0.40/ -0.40 len 100.00
 5 reward  -0.40/ -0.40/ -0.40 len 100.00
 6 reward  -0.40/ -0.40/ -0.40 len 100.00
 7 reward  -0.40/ -0.40/ -0.40 len 100.00
 8 reward  -0.40/ -0.40/ -0.40 len 100.00
 9 reward  -0.40/ -0.40/ -0.40 len 100.00
Saved checkpoint 10 at /content/drive/MyDrive/GitHub/INM363-Project/model_checkpoints/ppo/easy_no_reward/checkpoint_000010
10 reward  -0.40/ -0.40/ -0.40 len 100.00
11 reward  -0.40/ -0.40/ -0.40 len 100.00
12 reward  -0.40/ -0.40/ -0.40 len 100.00
13 reward  -0.40/ -0.40/ -0.40 len 100.00
14 reward  -0.40/ -0.40/ -0.40 len 100.00
15 reward  -0.40/ -0.40/ -0.40 len 100.00
16 reward  -0.40/ -0.40/ -0.40 len 100.00
17 reward  -0.40/ -0.40/ -0.40 len 100.00
18 reward  -0.40/ -0.40/ -0.40 len 100.00

# save results file

In [ ]:
from pathlib import Path 

fname = chkpt_root + '/result.csv'
fpath = Path(fname)
fpath.parent.mkdir(parents=True, exist_ok = True)
results_df.to_csv(fpath)
print(f"Saved results file to {fname}")


In [11]:
results_df.head()

,checkpoint,eps_reward_min,eps_reward_mean,eps_reward_max,eps_len_mean,episodes_this_iter
0,1.0,-0.8,-0.800000,-0.800,200.000000,20.0
1,2.0,-0.8,-0.128537,9.730,196.780488,21.0
2,3.0,-0.8,0.315328,9.954,190.734375,23.0
3,4.0,-0.8,0.464616,9.986,189.116279,22.0
4,5.0,-0.8,0.337570,9.986,190.640000,20.0


In [13]:
results_df.tail(100)

/usr/local/lib/python3.7/dist-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,checkpoint,eps_reward_min,eps_reward_mean,eps_reward_max,eps_len_mean,episodes_this_iter
300,301.0,-0.8,0.34498,9.998,183.80,21.0
301,302.0,-0.8,0.34430,9.998,183.97,21.0
302,303.0,-0.8,0.23820,9.998,185.49,21.0
303,304.0,-0.8,0.18792,9.998,185.56,22.0
304,305.0,-0.8,-0.01840,9.994,189.63,21.0
...,...,...,...,...,...,...
395,396.0,-0.8,0.20980,9.982,185.09,24.0
396,397.0,-0.8,0.19952,9.982,185.16,21.0
397,398.0,-0.8,0.32634,9.982,183.46,21.0
398,399.0,-0.8,0.33542,9.970,183.69,21.0
